## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
import sys
sys.path.append("C:/Users/c_tal/Desktop")
from api_keys import g_key
g_key = g_key

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/c_tal/Desktop/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.71,79,100,21.05,moderate rain
1,1,Hithadhoo,MV,-0.6000,73.0833,82.40,70,68,13.04,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,57.31,44,0,18.41,clear sky
3,3,Gat,IL,31.6100,34.7642,49.41,65,0,4.52,clear sky
4,4,Ushuaia,AR,-54.8000,-68.3000,53.26,50,40,11.50,scattered clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Get the maximum temperature. (Must do this if max temp is below 0 or error will return)
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,76.71,79,100,21.05,moderate rain
1,1,Hithadhoo,MV,-0.6000,73.0833,82.40,70,68,13.04,broken clouds
5,5,Souillac,MU,-20.5167,57.5167,76.35,88,40,4.61,scattered clouds
21,21,Bandarbeyla,SO,9.4942,50.8122,78.94,61,100,11.39,overcast clouds
24,24,Vaini,TO,-21.2000,-175.2000,84.36,84,75,11.50,broken clouds
35,35,Mataura,NZ,-46.1927,168.8643,76.48,34,5,9.78,clear sky
36,36,Bambous Virieux,MU,-20.3428,57.7575,76.21,88,40,4.61,scattered clouds
37,37,Morant Bay,JM,17.8815,-76.4093,80.22,84,26,7.36,scattered clouds
38,38,Busselton,AU,-33.6500,115.3333,84.16,40,34,11.74,scattered clouds
40,40,Maraa,BR,-1.8333,-65.3667,75.31,95,85,3.06,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# No empty rows - they are all 174

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,76.71,moderate rain,-23.1203,-134.9692,
1,Hithadhoo,MV,82.40,broken clouds,-0.6000,73.0833,
5,Souillac,MU,76.35,scattered clouds,-20.5167,57.5167,
21,Bandarbeyla,SO,78.94,overcast clouds,9.4942,50.8122,
24,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
35,Mataura,NZ,76.48,clear sky,-46.1927,168.8643,
36,Bambous Virieux,MU,76.21,scattered clouds,-20.3428,57.7575,
37,Morant Bay,JM,80.22,scattered clouds,17.8815,-76.4093,
38,Busselton,AU,84.16,scattered clouds,-33.6500,115.3333,
40,Maraa,BR,75.31,overcast clouds,-1.8333,-65.3667,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False)
hotel_df.count()

City                   174
Country                174
Max Temp               174
Current Description    174
Lat                    174
Lng                    174
Hotel Name             174
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))